In [1]:
import requests
import json
import csv

In [2]:
def getBlockHeaders(block_height_start,block_height_end,file_name):
    # bitmonerod is running on the localhost and port of 18082
    url = "http://localhost:18081/json_rpc"

    # standard json header
    headers = {'content-type': 'application/json'}

    # the block to get
    # bitmonerod' procedure/method to call
    rpc_input = {
           "method": "getblockheadersrange",
           "params": {"start_height": block_height_start, "end_height":block_height_end }
    }

    # add standard rpc values
    rpc_input.update({"jsonrpc": "2.0", "id": "0"})

    # execute the rpc request
    response = requests.post(
        url,
        data=json.dumps(rpc_input),
        headers=headers)

    # the response will contain binary blob. For some reason
    # python's json encoder will crash trying to parse such
    # response. Thus, its better to remove it from the response.
    response_json_clean = json.loads(
                            "\n".join(filter(
                                lambda l: "blob" not in l, response.text.split("\n")
                            )))

    # pretty print json output
    with open(file_name, "w") as text_file:
        text_file.write(json.dumps(response_json_clean, indent=4))

In [3]:
getBlockHeaders(1,1000,"json/output.txt")

In [4]:
with open('json/output.txt') as json_data:
    d = json.load(json_data)

In [5]:
with open('csv/output.csv','wb') as f:
    w = csv.DictWriter(f,
                      fieldnames='nonce reward hash timestamp block_weight major_version cumulative_difficulty '
                      'height difficulty depth long_term_weight prev_hash pow_hash orphan_status block_size '
                      'minor_version num_txes'.split(),
                      extrasaction='ignore')
    w.writeheader()
    w.writerows(d['result']['headers'])